# 如何分析 CANN 模型？

**目标**：本指南结束时，您将能够使用能量景观图来可视化 CANN 动力学。

**预计阅读时间**：12 分钟

---

## 介绍

运行 CANN 模拟后，您需要**可视化**结果以理解网络的行为。凸起是否正确形成？它是否跟踪刺激？活动是否稳定？

**Analyzer** 模块提供了专为 CANN 动力学设计的可视化工具。本指南重点关注最重要的可视化方法：**能量景观图**。

您将学习：
1. 创建静态能量景观图
2. 生成动画可视化
3. 比较不同模型（CANN 与 CANN 加 SFA）
4. 使用 PlotConfig 系统实现一致的样式

## 什么是能量景观？

**能量景观**显示神经活动如何随时间演变：
- **X 轴**：神经元位置（对于 1D CANN，这是从 -π 到 π 的角位置）
- **Y 轴**：神经活动或突触输入
- **动画**：活动模式的时间演变

此可视化非常适合观察：
- **凸起形成**：是否出现局部化的活动凸起？
- **跟踪**：凸起是否跟随外部刺激？
- **稳定性**：凸起是否保持其形状和位置？

## PlotConfig 系统

在深入示例之前，请了解可视化架构。

该库使用 **PlotConfig** 数据类来管理绘图设置。这提供了：
- **一致性**：不同绘图类型之间使用相同的参数
- **可重用性**：保存和重用配置
- **类型安全**：清晰的参数名称和验证


In [ ]:
from canns.analyzer.plotting import PlotConfigs

# 为动画创建配置
config = PlotConfigs.energy_landscape_1d_animation(
    time_steps_per_second=100,  # 模拟速度（100 步/秒）
    fps=20,                     # 动画帧速率
    title='My CANN Simulation',
    xlabel='Position (rad)',
    ylabel='Activity',
    save_path='output.gif',
    show=False                  # 不显示，只保存
)

**为什么使用 PlotConfig？** 与其向函数传递 10 多个参数，不如配置一次并重用。这在创建多个类似绘图时特别有价值。

## 完整工作流：从模型到可视化

让我们构建一个完整的示例，创建、模拟和可视化一个 CANN：


In [ ]:
import brainstate
from canns.models.basic import CANN1D
from canns.task.tracking import SmoothTracking1D
from canns.analyzer.plotting import PlotConfigs, energy_landscape_1d_animation

# 1. 设置
brainstate.environ.set(dt=0.1)

# 2. 创建模型
cann = CANN1D(num=512)
cann.init_state()

# 3. 创建任务
task = SmoothTracking1D(
    cann_instance=cann,
    Iext=(1.0, 0.75, 2.0, 1.75, 3.0),
    duration=(10.0, 10.0, 10.0, 10.0),
    time_step=brainstate.environ.get_dt(),
)
task.get_data()

# 4. 运行模拟
def run_step(t, inputs):
    cann(inputs)
    return cann.u.value, cann.r.value

us, rs = brainstate.transform.for_loop(
    run_step,
    task.run_steps,
    task.data,
    pbar=brainstate.transform.ProgressBar(10)
)

print("模拟完成！现在进行可视化...")

# 5. 创建可视化配置
config = PlotConfigs.energy_landscape_1d_animation(
    time_steps_per_second=100,
    fps=20,
    title='CANN1D Smooth Tracking',
    xlabel='Position (rad)',
    ylabel='Activity',
    repeat=True,
    save_path='cann1d_tracking.gif',
    show=False
)

# 6. 生成动画
energy_landscape_1d_animation(
    data_sets={
        'u': (cann.x, us),      # 随时间变化的突触输入
        'Iext': (cann.x, task.data)  # 外部刺激
    },
    config=config
)

print("动画已保存到 cann1d_tracking.gif")

**第 6 步发生了什么**：
- `data_sets`：将标签映射到 `(positions, data)` 元组的字典
- `cann.x`：神经元位置（X 轴值）
- `us`：随时间变化的活动（形状为 `[time_steps, num_neurons]`）
- `task.data`：用于比较的原始输入刺激

动画将显示两条曲线：
- **蓝色 (u)**：CANN 的内部活动
- **橙色 (Iext)**：外部输入刺激

您将看到活动凸起跟踪移动刺激！

## 用于快速检查的静态绘图

对于快速检查而不生成完整动画：


In [ ]:
from canns.analyzer.plotting import PlotConfigs, energy_landscape_1d

# 静态绘图配置
config_static = PlotConfigs.energy_landscape_1d(
    time_steps_per_second=100,
    title='Activity at t=200',
    xlabel='Position (rad)',
    ylabel='Activity',
    save_path='snapshot_t200.png',
    show=True  # 显示绘图
)

# 绘制特定时间步的活动（例如，t=200）
energy_landscape_1d(
    data_sets={'Activity': (cann.x, us[200])},  # 单个时间步
    config=config_static
)

**用例**：快速检查模型是否在特定时间达到稳定状态。

## 比较模型：CANN vs. CANN-SFA

一种强大的可视化技术是**并排比较不同的模型**。让我们比较标准 CANN 与包含**尖峰频率适应 (SFA)** 的 CANN：


In [ ]:
import brainstate
from canns.models.basic import CANN1D, CANN1D_SFA
from canns.task.tracking import SmoothTracking1D
from canns.analyzer.plotting import PlotConfigs, energy_landscape_1d_animation

brainstate.environ.set(dt=0.1)

# 创建两个模型
cann_standard = CANN1D(num=512)
cann_sfa = CANN1D_SFA(num=512, tau_v=50.0)  # 带适应时间常数的 SFA

cann_standard.init_state()
cann_sfa.init_state()

# 两个模型使用相同的任务
task = SmoothTracking1D(
    cann_instance=cann_standard,  # 使用一个模型作为参考
    Iext=(1.0, 0.75, 2.0, 1.75, 3.0),
    duration=(10.0, 10.0, 10.0, 10.0),
    time_step=0.1,
)
task.get_data()

# 模拟标准 CANN
def step_standard(t, inputs):
    cann_standard(inputs)
    return cann_standard.r.value

rs_standard = brainstate.transform.for_loop(
    step_standard,
    task.run_steps,
    task.data,
    pbar=brainstate.transform.ProgressBar(10)
)

# 模拟 CANN-SFA
def step_sfa(t, inputs):
    cann_sfa(inputs)
    return cann_sfa.r.value

# 为了公平比较而重新初始化
cann_sfa.init_state()
rs_sfa = brainstate.transform.for_loop(
    step_sfa,
    task.run_steps,
    task.data,
    pbar=brainstate.transform.ProgressBar(10)
)

# 在一个动画中可视化两者
config_comparison = PlotConfigs.energy_landscape_1d_animation(
    time_steps_per_second=100,
    fps=20,
    title='CANN vs CANN-SFA: Oscillatory Tracking',
    xlabel='Position (rad)',
    ylabel='Activity',
    repeat=True,
    save_path='comparison_standard_vs_sfa.gif',
    show=False
)

energy_landscape_1d_animation(
    data_sets={
        'CANN (Standard)': (cann_standard.x, rs_standard),
        'CANN-SFA': (cann_sfa.x, rs_sfa),
        'Input': (cann_standard.x, task.data)
    },
    config=config_comparison
)

print("比较动画已保存！")

**您将观察到**：
- **标准 CANN**：凸起紧密跟踪输入
- **CANN-SFA**：凸起可能显示振荡行为或由于适应导致的延迟跟踪
- **洞察**：SFA 引入了历史相关的动力学，可见为不同的跟踪模式

这种比较帮助您理解模型变体在相同条件下的不同行为。

## 自定义可视化

### 调整动画速度


In [ ]:
config_slow = PlotConfigs.energy_landscape_1d_animation(
    time_steps_per_second=50,   # 每秒显示 50 步（较慢）
    fps=10,                     # 每秒 10 帧（更平滑）
    ...
)

**经验法则**：`time_steps_per_second / fps` = 每帧步数
- 比率越高 = 动画越快
- 比率越低 = 动画越慢，细节越丰富

### 改变外观


In [ ]:
config_styled = PlotConfigs.energy_landscape_1d_animation(
    figsize=(10, 4),            # 更宽的图形
    title='Custom Styled Plot',
    xlabel='Angular Position',
    ylabel='Firing Rate (Hz)',
    title_fontsize=14,
    save_path='styled_output.gif',
    show=False
)

### 保存与显示


In [ ]:
# 仅保存（用于批处理）
config_save = PlotConfigs.energy_landscape_1d_animation(
    save_path='output.gif',
    show=False
)

# 仅显示（用于交互式探索）
config_display = PlotConfigs.energy_landscape_1d_animation(
    save_path=None,
    show=True
)

# 两者
config_both = PlotConfigs.energy_landscape_1d_animation(
    save_path='output.gif',
    show=True
)

## 更多分析工具

本指南重点关注能量景观，但分析器模块包含许多其他工具：

- **调谐曲线**：神经元选择性绘图
- **放电场**：空间响应图
- **凸起拟合**：提取凸起位置和宽度
- **尖峰图**：基于尖峰的模型的栅栏图

这些内容在[完整详情：模型分析器](link-to-full-details)部分有详细介绍。

## 后续步骤

现在您可以可视化 CANN 动力学，您已准备好：

1. **[分析实验数据](05_how_to_analyze_experimental_data.md)** - 将类似的技术应用于真实神经记录
2. **[探索模型分析器](link-to-core-concepts-analyzer)** - 了解所有可用的分析方法
3. **[完整可视化 API](link-to-full-details-analyzer)** - PlotConfig 和所有绘图函数的完整参考

---

**快速参考**：


In [ ]:
# 标准可视化工作流
from canns.analyzer.plotting import PlotConfigs, energy_landscape_1d_animation

# 1. 配置
config = PlotConfigs.energy_landscape_1d_animation(
    time_steps_per_second=100,
    fps=20,
    save_path='output.gif'
)

# 2. 可视化
energy_landscape_1d_animation(
    data_sets={'Activity': (positions, activity_over_time)},
    config=config
)

---

*有问题？查看[PlotConfig 指南](link-to-plotconfig-detail)或[GitHub 讨论](https://github.com/routhleck/canns/discussions)。*